In [ ]:
-- We identified five basic entities in our schema:

-- 1. Hospital
-- 2. Location
-- 3. Hospital Quality
-- 4. Weekly Report
-- 5. Bed Report

-- We chose these entities to effectively separate information like hospital details and location
-- from weekly reports and quality ratings. This approach reduces redundancy by
-- making sure that each piece of information is stored only once.


-- Table for hospital information (Hospital Table)
-- This table represents the 'Hospital' entity and stores information about each hospital.
-- We use 'hospital_pk' as the primary key to uniquely identify each hospital.
-- The 'location_id' is a foreign key referencing the 'location' table, linking each hospital to its location.
CREATE TABLE hospital (
    hospital_pk VARCHAR(50) PRIMARY KEY,
    hospital_name TEXT NOT NULL,
    location_id INTEGER REFERENCES location(id) ON DELETE SET NULL,
    latitude NUMERIC(9, 6),
    longitude NUMERIC(9, 6),
    type_of_hospital TEXT,
    ownership TEXT,
    provides_emergency_services BOOLEAN
);

-- Table to store unique location information (Location Table)
-- The 'Location' entity stores unique geographical information to avoid redundancy when multiple hospitals
-- share the same location details, which helps to minimize duplication.
CREATE TABLE location (
    id SERIAL PRIMARY KEY,
    city TEXT,
    state CHAR(2) NOT NULL,
    zip_code CHAR(10),
    fips_code VARCHAR(10)
);

-- Table to store the quality ratings of each hospital (Hopsital Quality)
-- This table keeps track of quality ratings for each hospital over time.
-- 'facility_id' links back to the 'hospital' table.
-- We can see how a hospital's rating changes by date through this table. 
CREATE TABLE hospital_quality (
    id SERIAL PRIMARY KEY,
    facility_id VARCHAR(50) REFERENCES hospital(hospital_pk) ON DELETE CASCADE,
    quality_rating INTEGER CHECK (quality_rating BETWEEN 1 AND 10), 
    rating_date DATE NOT NULL                                   
);

-- Table to store weekly data reports for each hospital (Weekly information)
-- This table stores weekly reports for each hospital.
-- 'hospital_weekly_id' links to the 'hospital' table.
-- We ensure there's only one report per hospital each week.
CREATE TABLE weekly_report (
    id SERIAL PRIMARY KEY,
    hospital_weekly_id VARCHAR(50) REFERENCES hospital(hospital_pk) ON DELETE CASCADE,
    collection_week DATE NOT NULL,                                -- Week of data collection
    UNIQUE(hospital_wid, collection_week)                         -- Ensure one entry per hospital per week
);

-- Table to store bed-related information (Info about beds)
-- This table holds data about bed usage for each weekly report.
-- It includes the type of bed and averages of beds available and occupied.
-- 'weekly_bed_report_id' links to the 'weekly_report' table.
CREATE TABLE bed_report (
    id SERIAL PRIMARY KEY,
    weekly_bed_report_id INTEGER REFERENCES weekly_report(id) ON DELETE CASCADE,
    bed_type TEXT NOT NULL,                                       -- Type of bed (e.g., adult, pediatric, ICU)
    beds_available_7_day_avg NUMERIC(10, 2),                      -- Avg. beds available
    beds_occupied_7_day_avg NUMERIC(10, 2),                       -- Avg. beds occupied
    covid_patients_7_day_avg NUMERIC(10, 2),                      -- COVID patients in these beds
    UNIQUE(weekly_report_id, bed_type)                            -- Ensure unique entry per bed type per report
);

-- Why we designed it this way:

-- We separated data into different tables to keep things organized and avoid duplicates:

-- - The 'hospital' table stores each hospital's info once.
-- - The 'location' table holds location details that can be shared by multiple hospitals.
-- - The 'hospital_quality' table tracks quality ratings over time without repeating hospital info.
-- - The 'weekly_report' and 'bed_report' tables store time-based data linked to hospitals.

-- By linking tables with foreign keys, we connect related data without storing the same information multiple times.
-- This makes our database efficient and easier to maintain.

Relationships:

| Table | Primary Key | Foreign Key | Relationship |
| --- | --- | --- | --- |
| `location` | `id` | None | Standalone table for unique locations |
| `hospital` | `hospital_pk` | `location_id` references `location(id)` | Links each hospital to a unique location |
| `hospital_quality` | `id` | `facility_id` references `hospital(hospital_pk)` | Links each quality rating to a hospital |
| `weekly_report` | `id` | `hospital_wid` references `hospital(hospital_pk)` | Links each weekly report to a hospital |
| `bed_report` | `id` | `weekly_report_id` references `weekly_report(id)` | Links each bed report to a specific weekly report. Each bed report entry represents details for a specific bed type within that weekly report. |